In [1]:
import pandas as pd
import os
import gc

In [2]:
path = r'G:\py_workspace\20191103_ptt_public_opinion\data\verified-source\data\\'

In [3]:
def article_dt_rng_check(art_df):
    dt = art_df.date.str[:10]
    rs = dt.groupby(dt).count()
    #avoid backup articles & sys err's future time
    fil_rs = rs[rs > (rs.mean()/5)].sort_index()
    strt_dt = fil_rs.index[0]
    end_dt = fil_rs.index[-1]
    return [strt_dt, end_dt]

In [4]:
#board_nm = 'Gossiping' 
board_nm = 'HatePolitics'

file_lst = []

for x in os.listdir(path):
    if 'article' in x and board_nm in x:
        print(x)
        df = pd.read_csv(path + x)
        strt_dt, end_dt = article_dt_rng_check(df)
        file_lst.append([x, strt_dt, end_dt])
        file_df = pd.DataFrame(file_lst, columns=['file_nm', 'strt_dt', 'end_dt']).sort_values('strt_dt')

HatePolitics-0-4060_0618_article_20200105_133804.csv
HatePolitics-0-4062_article_20200105_133804.csv
HatePolitics-2600-4035_article_20200105_133804.csv
HatePolitics-2800-4285_201912_article_20200105_133804.csv
HatePolitics-3000-3986_1207_article_20200105_133804.csv


# data type transform

In [5]:
file_df.reset_index(drop=True, inplace=True)
file_df.strt_dt = pd.to_datetime(file_df.strt_dt)
file_df.end_dt = pd.to_datetime(file_df.end_dt)
gc.collect()

76

In [7]:
#seq merging...
#check end_dt >= next strt_dt
for x in file_df[:-1].itertuples():
    if  x.end_dt >= file_df.loc[x.Index + 1].strt_dt:
        print('pass')
    else:
        raise ValueError('remove invalid record or complement insufficient data')

pass
pass
pass
pass


In [8]:
file_df

,file_nm,strt_dt,end_dt
0,HatePolitics-0-4060_0618_article_20200105_1338...,2006-09-29,2019-06-18
1,HatePolitics-0-4062_article_20200105_133804.csv,2006-09-29,2019-10-15
2,HatePolitics-2600-4035_article_20200105_133804...,2019-10-01,2019-11-18
3,HatePolitics-3000-3986_1207_article_20200105_1...,2019-11-11,2019-12-08
4,HatePolitics-2800-4285_201912_article_20200105...,2019-11-30,2020-01-03


# merge file

In [ ]:
main_art_df = pd.DataFrame()
main_reply_df = pd.DataFrame()
df_type_lst = ['reply', 'article']
#one more loop for df type
for df_type in df_type_lst: 
    for x in range(file_df.shape[0])[::-1]:
        ori_f_nm = '_'.join(file_df.loc[x].file_nm.split('_')[:-3])

        for file in os.listdir(path):
            if ori_f_nm in file:
                if df_type in file:
                    print(file)
                    temp_art_df = pd.read_csv(path + file)

                    if main_art_df.empty:
                        main_art_df = temp_art_df
                    else:
                        main_art_df = temp_art_df[~temp_art_df.article_id.isin(main_art_df.article_id)].append(main_art_df)
        del temp_art_df
        gc.collect()
        
    dt = pd.to_datetime(main_art_df.date)
    output_strt_dt = str(dt.min())[:10]
    output_end_dt = str(dt.max())[:10]
    #comma has been removed from step1
    main_art_df.to_csv(path + '{}_{}_intg_{}_{}.csv'.format(board_nm, df_type, output_strt_dt, output_end_dt), index=False)

HatePolitics-2800-4285_201912_reply_20200105_133804.csv
HatePolitics-3000-3986_1207_reply_20200105_133804.csv
HatePolitics-2600-4035_reply_20200105_133804.csv
HatePolitics-0-4062_reply_20200105_133804.csv
HatePolitics-0-4060_0618_reply_20200105_133804.csv
HatePolitics-2800-4285_201912_article_20200105_133804.csv


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


HatePolitics-3000-3986_1207_article_20200105_133804.csv
HatePolitics-2600-4035_article_20200105_133804.csv
HatePolitics-0-4062_article_20200105_133804.csv
HatePolitics-0-4060_0618_article_20200105_133804.csv


In [ ]:
#once one type
#elif 'reply' in file:
#    print(file)
#    temp_reply_df = pd.read_csv(path + file)
#    
#    if main_reply_df.empty:
#        main_reply_df = temp_reply_df
#    else:
#        main_reply_df = temp_reply_df[~temp_reply_df.article_id.isin(main_reply_df.article_id)].append(main_reply_df)

In [ ]:
#df_1['rnk'] = 'df_1'
#df_2['rnk'] = 'df_2'
#art_df = df_1[~df_1.article_id.isin(df_2.article_id)].append(df_2)